In [3]:
# Importa la biblioteca pandas para la manipulación de datos en formato tabular
import pandas as pd
# Importa la biblioteca os para interactuar con el sistema operativo
import os
# Importa SparkSession de PySpark para iniciar una sesión de Spark
from pyspark.sql import SparkSession
# Importa funciones
from pyspark.sql.functions import lpad

In [4]:
# Crea una sesión de Spark con el nombre de la aplicación "Conversion_from_CSV_to_Parquet", habilitando el soporte para Hive, y la inicializa o reutiliza si ya existe
spark = SparkSession \
    .builder \
    .appName("Conversión de CSV a Parquet") \
    .enableHiveSupport() \
    .getOrCreate()

24/10/22 00:20:36 WARN Utils: Your hostname, felipe-Nitro-AN515-44 resolves to a loopback address: 127.0.1.1; using 192.168.15.10 instead (on interface wlp5s0)
24/10/22 00:20:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/22 00:20:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/22 00:20:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
# Inicia una sesión de Spark con el nombre de la aplicación "CSV to Parquet"
spark = SparkSession.builder.appName("CSV to Parquet").getOrCreate()

# Define la ruta donde se encuentran los archivos CSV
landing_zone = '/home/felipe/despliegue_analytica/files_csv/'

# Lista todos los archivos en el directorio especificado
files = os.listdir(landing_zone)

# Bucle para transformar todos los archivos CSV en formato Parquet
for filename in files:
    if filename.endswith('.csv'):
        # Lee el archivo CSV con la configuración de encabezados, detección automática de esquema y separador de comas
        csv = spark.read.csv(f'{landing_zone}/{filename}', header=True, inferSchema=True, sep=',')

        # Añade ceros a la izquierda para los prefijos de código postal en ciertos archivos específicos
        if filename == 'olist_customers_dataset.csv':
            csv = csv.withColumn('customer_zip_code_prefix', lpad(csv.customer_zip_code_prefix, 5, '0'))
        elif filename == 'olist_sellers_dataset.csv':
            csv = csv.withColumn('seller_zip_code_prefix', lpad(csv.seller_zip_code_prefix, 5, '0'))
        elif filename == 'olist_geolocation_dataset.csv':
            csv = csv.withColumn('geolocation_zip_code_prefix', lpad(csv.geolocation_zip_code_prefix, 5, '0'))

        # Modifica el nombre del archivo para que tenga la extensión .parquet
        parquet_name = filename.replace('csv', 'parquet')

        # Guarda el archivo en formato Parquet en la carpeta "files_parquet"
        csv.write.parquet(f'/home/felipe/despliegue_analytica/files_parquet/{parquet_name}', mode='overwrite')

# Imprime un mensaje indicando que los archivos CSV se han convertido y guardado como Parquet en la carpeta "olist"
print("Arquivos CSV convertidos para Parquet e salvos em /olist/")